This code is for training a neural network into learning how to play the atari game from the gym API.

The process will consist on loading multiple datasets that must be in the working directory of this notebook, concatenating them to create a dataset and parsing the dataset to the network.

The user is free to change the number of epochs and the parameters of the neural network. However, I do not advise to change the parameters of network since the training accuracy is most likely to reduce considerably. The most sensitive parameter is the loss. All the possible options of loss were tried and none of them gave a result as good as binary_crossentropy.

In [3]:
import os
import gym
import cv2
import argparse
import sys, glob
import numpy as np
import pandas as pd
import pdb
import keras
from keras import backend as K
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, Reshape
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam, Adamax, RMSprop
from keras.layers.advanced_activations import PReLU
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Activation, Dropout, Flatten
from keras.layers.convolutional import UpSampling2D, Convolution2D
IMAGE_LENGTH = int(33600)
epochs = 1


Using TensorFlow backend.


In [0]:
# This code is for loading the images of the dataset given a path.
# IMPORTANT: The dataset most contain flattenned image of leng 33600 and
# before the first element of this vector there must be the feature of that
# sample. That feature is the keyboard pressed by the user while playing and 
# it has to be an integer between 0 and 8.
def load_data(path):
    #pdb.set_trace()
    df = pd.read_csv(path, sep = '.', header = None)
    letters = df[(df.index % (IMAGE_LENGTH+1)==0)].values.tolist()
    images = df[(df.index % (IMAGE_LENGTH+1)!=0)].values.tolist()
    n =IMAGE_LENGTH
    final = [images[i * n:(i + 1) * n] for i in range((len(images) + n - 1) // n )]  
    #pdb.set_trace()
    return letters, final

In [0]:
# Prepare first dataset 
letter, final = load_data('/content/drive/My Drive/DATOS/EIT/NICE ACADEMIC/AI/project/dataset_thomas_v1.txt')
for i in range (0, len(final)):
    final[i] = np.concatenate(final[i])
#pdb.set_trace()

letter = np.concatenate(letter)
final = np.array(final)
letter=letter.reshape(1, len(letter))
letter = letter[0, :]

In [0]:
# Prepare  second dataset 
letter2, final2 = load_data('/content/drive/My Drive/DATOS/EIT/NICE ACADEMIC/AI/project/dataset_thomas_v2.txt')
for i in range (0, len(final2)):
    final2[i] = np.concatenate(final2[i])
#pdb.set_trace()

letter2 = np.concatenate(letter2)
final2 = np.array(final2)
letter2=letter2.reshape(1, len(letter2))
letter2 = letter2[0, :]

In [0]:
# Prepare third dataset
# Prepare  second dataset 
letter3, final3 = load_data('/content/drive/My Drive/DATOS/EIT/NICE ACADEMIC/AI/project/dataset_thomas_v3.txt')
for i in range (0, len(final3)):
    final3[i] = np.concatenate(final3[i])
#pdb.set_trace()

letter3 = np.concatenate(letter3)
final3 = np.array(final3)
letter3=letter3.reshape(1, len(letter3))
letter3 = letter3[0, :]

In [0]:
# Prepare third dataset
# Prepare  second dataset 
letter4, final4 = load_data('/content/drive/My Drive/DATOS/EIT/NICE ACADEMIC/AI/project/dataset_haider_v1.txt')
for i in range (0, len(final4)):
    final4[i] = np.concatenate(final4[i])
#pdb.set_trace()

letter4 = np.concatenate(letter4)
final4 = np.array(final4)
letter4=letter4.reshape(1, len(letter4))
letter4 = letter4[0, :]

In [0]:
final = np.concatenate((final, final2, final3, final4))

In [0]:
letter = np.concatenate((letter, letter2, letter3, letter4))

In [11]:
# This is printing the number of images we have of gameplay. 
# This number should be much higher to properly train the network however 13 thousand
# already correspond to above 30 minutes of game play.
print('We have a total of {} frames ready for training!'.format(final.shape[0]))

We have a total of 13233 frames ready for training!


In [0]:
# Split dataset using a rule of 0.7
train_ratio = 0.7

n_train_samples = int(len(final) * train_ratio)
x_train, y_train = final[:n_train_samples], letter[:n_train_samples]
x_val, y_val = final[n_train_samples:], letter[n_train_samples:]


In [0]:
# The number of classes are all the possible keys a user can press during the game
# Even not pressing any key is allowed.
num_classes = 9
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)


In [16]:
# Input shape will be 33600 since that is the image size.
# This neural network has only 2 layers. The user is free to try new layers.
# An experiment with eleven layers was performed but no improvement was observed.
model = Sequential()
model.add(Dense(150, activation='tanh', input_shape=(33600,)))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])
#pdb.set_trace()
history = model.fit(x_train, y_train,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val))
score = model.evaluate(x_val, y_val, verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])
# This code saves the model which will be needed for letting the network play the game
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_weights.h5")
print("Saved model to disk")


Train on 9263 samples, validate on 3970 samples
Epoch 1/1
9263/9263 [==============================] - 4s 426us/step - loss: 0.1885 - accuracy: 0.9201 - val_loss: 0.6235 - val_accuracy: 0.8205
Validation loss: 0.6235493505631646
Validation accuracy: 0.8205432891845703
Saved model to disk
